# Route Planner: A Prolog Application with LangChain Integration

## Overview
The `routes.pl` application is a travel route planning system implemented in Prolog that demonstrates logical reasoning capabilities integrated with LangChain. It serves as an example of how Prolog's declarative programming paradigm can enhance LLM applications with powerful reasoning.

## Key Features
- **Transport Network Modeling:** Intuitively defines connections between cities with various transport types (train, plane, ferry).  
- **Route Finding:** Discovers all possible routes between origin and destination cities.  
- **Optimization:** Calculates fastest and cheapest routes based on time and cost parameters.  
- **Constraint Handling:** Applies constraints like maximum time, maximum cost, and number of connections.  
- **Transport Preferences:** Filters routes based on preferred transport types.  
- **Comprehensive Query Interface:** Provides predicates for different query types and formats.  

## Technical Highlights

- Uses Prolog's backtracking to explore all possible routes.  
- Implements cycle detection to prevent infinite loops.  
- Provides helper predicates for visualizing and analyzing routes.  
- Structures data for easy integration with Python.  
- Returns results in formats compatible with LangChain Runnable and Tool interfaces.  

## Integration with LangChain
When integrated with LangChain or LangGraph, this application enables:

- Natural language queries about travel routes.  
- Reasoning about optimal travel plans based on constraints.  
- Explanations of why certain routes are preferred.  
- Step-by-step travel planning with LLM guidance.  

## How to use this notebook:
1. Run the setup cell to install SWI-Prolog and required packages
2. Run each cell in sequence to see the route planning in action
3. Try modifying the queries to explore different routes

## Setup

**Prerequisites**

- Python 3.10 or later
- SWI-Prolog installed on your system
- The following python libraries installed:
    - langchain 0.3.0 or later
    - janus-swi 1.5.0 or later
    - pydantic 2.0.0 or later

The Prolog interfase with LangChain can be installed using pip:

**&ensp;&ensp;&ensp;&ensp;&ensp;pip install langchain-prolog**

<br>

---


**Note:** This notebook installs SWI-Prolog and the required Python packages, and if you choose to use Ollama as the provider for the chat model, it then downloads and uses LLama running directly in Google Colab instead of requiring an API key.
This setup process takes a few minutes. For better performance, consider running this notebook locally and using an API based chat model.

If you choose to run it in Google Colab with an API-based chat model, you must provide your API key, preferably as an environment variable.

---

In [ ]:
#@title Run this cell to setup SWI-Prolog and the required Python packages. Click on > to view/hide the script. {display-mode: "form"}

from IPython.display import display, HTML, clear_output
import os
import subprocess
import time

# Function to create progress bar
def show_progress(step, total, progress, status="Working..."):
  bar_style = f"""
  <div style="
      width: 100%;
      height: 20px;
      background-color: #f1f1f1;
      border-radius: 5px;
      margin: 10px 0;
  ">
      <div style="
          width: {progress}%;
          height: 100%;
          background-color: #4CAF50;
          border-radius: 5px;
          text-align: center;
          line-height: 20px;
          color: white;
      ">
          {int(progress)}%
      </div>
  </div>
  <div style="text-align: center; margin-bottom: 15px; font-weight: bold;">
      {status} ({step}/{total})
  </div>
  """
  clear_output(wait=True)
  display(HTML(bar_style))

total_steps = 3

# SWI-Prolog installation
show_progress(1, total_steps, 0,  "Installing SWI-Prolog")
!apt-add-repository ppa:swi-prolog/stable -y
!apt-get update
!apt-get install -y swi-prolog

# SWI-Prolog environment setup
swipl_home = subprocess.check_output(
    "swipl -g \"current_prolog_flag(home, Home), writeln(Home)\" -t halt",
    shell=True
).decode().strip()
arch = subprocess.check_output("uname -m", shell=True).decode().strip()
os.environ["SWI_HOME_DIR"] = swipl_home
ld_library_path = f"{swipl_home}/lib/{arch}-linux"
if "LD_LIBRARY_PATH" in os.environ:
    os.environ["LD_LIBRARY_PATH"] = f"{ld_library_path}:{os.environ['LD_LIBRARY_PATH']}"
else:
    os.environ["LD_LIBRARY_PATH"] = ld_library_path

show_progress(2, total_steps, 60, "Installing langchain-prolog")
!pip install --quiet "langchain>=0.3.0"
!pip install --quiet "pydantic>=2.0.0"
!pip install --quiet "janus-swi>=1.5.0"
!pip install --quie langchain-prolog

show_progress(3, total_steps, 80, "Installing LangGraph")
!pip install --quiet "langgraph==0.2.74"
!pip install --quiet "langgraph-checkpoint==2.0.16"
!pip install --quiet "langgraph-sdk==0.1.53"
!pip install --quiet langchain-community

show_progress(3, total_steps, 100, "Installation completed")
time.sleep(2)

openai_installed = False
anthropic_installed = False
ollama_installed = False

try:
    import langchain_prolog
    from langchain_prolog import PrologConfig, PrologRunnable, PrologTool
    display(HTML(f"<div style='padding: 10px; background-color: #d4edda; border-radius: 5px; text-align: center;'><b>Success!</b> langchain-prolog {langchain_prolog.__version__} installed</div>"))
except ImportError as e:
    display(HTML(f"<div style='padding: 10px; background-color: #f8d7da; border-radius: 5px; text-align: center;'><b>Error:</b> {str(e)}</div>"))

def show_ai_message(response, height=200):
  """
  Display AI message in a scrollable container with better formatting.

  Args:
      response (str): The AI's text response
      height (int): Height of the scrollable area in pixels
  """
  # Escape HTML characters to prevent injection
  import html
  escaped_text = html.escape(response)

  # Create the HTML with the requested title format
  html_content = f"""
  <div style="margin: 20px 0;">
      <div style="
          font-weight: bold;
          margin-bottom: 10px;
          text-align: center;
          font-family: monospace;
          font-size: 16px;
          color: #2c3e50;
      ">================================== AI Message ==================================</div>
      <div style="
          height: {height}px;
          overflow-y: auto;
          border: 1px solid #ccc;
          border-radius: 4px;
          padding: 15px;
          background-color: #f9f9f9;
          font-family: monospace;
          white-space: pre-wrap;
          line-height: 1.5;
      ">
      {escaped_text}
      </div>
  </div>
  """

  display(HTML(html_content))

In [ ]:
#@title Run this cell to choose the Chat model provider. Click on > to view/hide the script. {display-mode: "form"}
import getpass
import os
from ipywidgets import widgets
from IPython.display import display

provider_widget = widgets.Dropdown(options=['Ollama', 'OpenAI', 'Anthropic'], description='Provider: ')
display(provider_widget)
provider = None

def on_button_clicked(b):
    global provider

    provider = provider_widget.value
    print(f"\n{provider} selected")

button = widgets.Button(description="Chose Provider")
button.on_click(on_button_clicked)
display(button)

In [ ]:
#@title Run this cell to setup the Chat model provider. Click on > to view/hide the script. {display-mode: "form"}
# These packages are not required by lanchain-prolog but are used by this notebook

import time
from IPython.display import display, HTML

if provider == "OpenAI" and not openai_installed:

  show_progress(1, 1, 0,  "Installing langchain-openai")
  if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

  !pip install --quiet langchain-openai
  openai_installed = True
  show_progress(1, 1, 100,  "OpenAI installed")

elif provider == "Anthropic" and not anthropic_installed:

  show_progress(1, 1, 0,  "Installing langchain-anthropic")
  if not os.environ.get("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

  !pip install --quiet langchain-anthropic
  anthropic_installed = True
  show_progress(1, 1, 100,  "Anthropic installed")

elif provider == "Ollama" and not ollama_installed:

  total_steps = 3
  show_progress(1, total_steps, 0,  "Installing langchain-ollama")
  !pip install --quiet langchain-ollama
  !pip install --quiet ollama

  show_progress(2, total_steps, 20,  "Installing Ollama")
  !curl -fsSL https://ollama.com/install.sh | sh

  # Create directory for Ollama
  !mkdir -p /root/.ollama

  # Start Ollama service in the background
  !nohup ollama serve > ollama.log 2>&1 &

  # Wait for Ollama to start
  time.sleep(10)

  # Check if Ollama is running
  !curl -s http://localhost:11434/api/tags

  show_progress(3, total_steps, 60,  "Downloading LLama")
  # Pull llama3.1-8b which supports function calling and fits in Colab's memory
  print("\nDownloading llama3.1-8b (this will take 5-10 minutes)...")
  !ollama pull llama3.1:8b

  # Verify the model is downloaded
  !ollama list

  ollama_installed = True
  show_progress(3, total_steps, 100,  "Ollama istalled")

time.sleep(2)
display(HTML(f"<div style='padding: 10px; background-color: #d4edda; border-radius: 5px; text-align: center;'><b>Success!</b> {provider} installed</div>"))

## The Prolog application

In [ ]:
#@title Click on > to view/hide the Prolog app. Run this cell to save it as a file.{display-mode: "form"}

%%writefile routes.pl
:- module(routes, [
    query_route/5,          % Main interface
    route_stats/2           % Route statistics
]).

% Transport connections with time (hours), cost (euros), and transport type
connection(london, paris, transport(train, 2.5, 150)).
connection(london, paris, transport(plane, 1.5, 200)).
connection(paris, rome, transport(plane, 2, 180)).
connection(paris, barcelona, transport(plane, 1.5, 120)).
connection(paris, madrid, transport(train, 2, 150)).
connection(barcelona, madrid, transport(train, 3, 80)).
connection(madrid, lisbon, transport(train, 4, 90)).
connection(rome, athens, transport(ferry, 8, 120)).
connection(rome, athens, transport(plane, 1.5, 160)).
connection(barcelona, rome, transport(plane, 2, 140)).

% Bidirectional connections. This is optional.
valid_connection(A, B, Transport) :-
    (   connection(A, B, Transport)
    ;   connection(B, A, Transport)
    ).

% Query type definitions
valid_query_type(all).        % All routes
valid_query_type(fastest).    % Fastest route only
valid_query_type(cheapest).   % Cheapest route only

% Main interface predicate
query_route(QueryType, From, To, Options, Results) :-
    must_be(atom, QueryType),
    must_be(atom, From),
    must_be(atom, To),
    must_be(list, Options),

    % Validate query type
    (   valid_query_type(QueryType)
    ->  true
    ;   throw(error(invalid_query_type(QueryType), _))
    ),

    % Execute query based on type
    execute_query(QueryType, From, To, Options, Results).

% Execute different types of queries
execute_query(QueryType, From, To, Options, Results) :-
    findall(
        Dict,
        (   route(From, To, Route, details(Time, Cost, Transport)),
            apply_options(Options, Time, Cost, Transport),
            maplist(transport_to_dict, Transport, TransportList),
            Dict = _{
                'route': Route,
                'time': Time,
                'cost': Cost,
                'transport': TransportList
            }
        ),
        AllRoutes
    ),
    % Select routes based on query type
    select_routes(QueryType, AllRoutes, Results).

% Select routes based on query type
select_routes(all, [Route|Rest], [Route|Rest]).
select_routes(fastest, Routes, [FastestRoute]) :-
    find_fastest_route(Routes, FastestRoute).
select_routes(cheapest, Routes, [CheapestRoute]) :-
    find_cheapest_route(Routes, CheapestRoute).

% Find fastest route
find_fastest_route([Route|Rest], FastestRoute) :-
    Route.get(time) = Time,
    find_fastest_route(Rest, Time, Route, FastestRoute).

find_fastest_route([], _, FastestSoFar, FastestSoFar).
find_fastest_route([Route|Rest], MinTime, CurrentBest, FastestRoute) :-
    Route.get(time) = Time,
    (   Time < MinTime
    ->  find_fastest_route(Rest, Time, Route, FastestRoute)
    ;   find_fastest_route(Rest, MinTime, CurrentBest, FastestRoute)
    ).

% Find cheapest route
find_cheapest_route([Route|Rest], CheapestRoute) :-
    Route.get(cost) = Cost,
    find_cheapest_route(Rest, Cost, Route, CheapestRoute).

find_cheapest_route([], _, CheapestSoFar, CheapestSoFar).
find_cheapest_route([Route|Rest], MinCost, CurrentBest, CheapestRoute) :-
    Route.get(cost) = Cost,
    (   Cost < MinCost
    ->  find_cheapest_route(Rest, Cost, Route, CheapestRoute)
    ;   find_cheapest_route(Rest, MinCost, CurrentBest, CheapestRoute)
    ).

% Route finding with constraints
route(From, To, Route, Details) :-
    find_route(From, To, [From], Route, [], Details).

% Base case: direct connection
find_route(From, To, _, [From,To], TransportList,
          details(Time, Cost, FinalTransport)) :-
    valid_connection(From, To, transport(Type, Time, Cost)),
    append(TransportList, [transport(Type, Time, Cost)], FinalTransport).

% Recursive case with cycle detection
find_route(From, To, Visited, [From|Route], AccTransport,
          details(TotalTime, TotalCost, FinalTransport)) :-
    valid_connection(From, Next, transport(Type, Time, Cost)),
    \+ member(Next, Visited),  % Prevent cycles
    \+ member(To, Visited),  % Prevent redudant paths
    length(Visited, Len),
    Len < 5,  % Limit path length to prevent excessive searching
    append(AccTransport, [transport(Type, Time, Cost)], NewTransport),
    find_route(Next, To, [Next|Visited], Route, NewTransport,
              details(RestTime, RestCost, FinalTransport)),
    TotalTime is Time + RestTime,
    TotalCost is Cost + RestCost.

% Apply filtering options
apply_options([], _, _, _) :- !.
apply_options(Options, Time, Cost, Transport) :-
    % Time constraint
    \+ (
        member(max_time(MaxTime), Options),
        Time > MaxTime
    ),
    % Cost constraint
    \+ (
        member(max_cost(MaxCost), Options),
        Cost > MaxCost
    ),
    % Changes constraint
    \+ (
        member(max_changes(MaxChanges), Options),
        length(Transport, Changes),
        Changes > MaxChanges + 1
    ),
    % Transport type constraint
    \+ (
        member(transport_type(PreferredTypeList), Options),
        has_nonpreferred_transport_type(Transport, PreferredTypeList)
    ).

% Helper predicate to check if route has required transport type
has_nonpreferred_transport_type(Transport, PreferredTypeList) :-
    member(transport(TransportType, _, _), Transport),
    \+ member(TransportType, PreferredTypeList).

% Convert transport term to dict
transport_to_dict(transport(Type, Time, Cost), Dict) :-
    Dict = _{
        'type': Type,
        'time': Time,
        'cost': Cost
    }.

% Route statistics
route_stats(From, To) :-
    findall(Details, route(From, To, _, Details), AllDetails),
    AllDetails \= [],
    findall(Time, member(details(Time,_,_), AllDetails), Times),
    findall(Cost, member(details(_,Cost,_), AllDetails), Costs),
    min_list(Times, MinTime),
    max_list(Times, MaxTime),
    sum_list(Times, TotalTime),
    length(Times, Count),
    AvgTime is TotalTime / Count,
    min_list(Costs, MinCost),
    max_list(Costs, MaxCost),
    sum_list(Costs, TotalCost),
    AvgCost is TotalCost / Count,
    format('~nRoute Statistics:~n'),
    format('Number of routes: ~w~n', [Count]),
    format('Time range: ~2f - ~2f hours (avg: ~2f)~n',
           [MinTime, MaxTime, AvgTime]),
    format('Cost range: €~2f - €~2f (avg: €~2f)~n',
           [MinCost, MaxCost, AvgCost]).

## Instantiation
The most important classes in langchain-prolog are: `PrologConfig`, `PrologRunnable` and `PrologTool`
- `PrologConfig` sets the configuration for the Prolog interpreter. The only mandatory field is the path to the Prolog script to be used.
- `PrologRunnable.create_schema` defines a Pydantic schema to be used to pass arguments to the Prolog predicates. It is optional, but recomended.
- `PrologTool` wraps the Prolog script with the interfase to LangChain/LangGraph. I supports all the methods of the `Tool` class and tracing capabilities of LangSmith.

In [ ]:
from langchain_prolog import PrologConfig, PrologRunnable, PrologTool

schema = PrologRunnable.create_schema('query_route', ['query_type', 'from', 'to', 'options', 'results'])

config = PrologConfig(
            rules_file='routes.pl',
            query_schema=schema,
            default_predicate="query_route"
        )

planner_tool = PrologTool(
    prolog_config=config,
    name="prolog_travel_planner",
    description="""
        Query travel routes using Prolog.
        Input have to be a dictionay like:
          {
            'query_type': 'all',
            'from': 'paris',
            'to': 'london',
            'options' [],
            'results': None
          }
        You always have to specify 4 parameters:
            - query_type: can be 'all', 'fastest' or 'cheapest'
            - from: the city where the travel starts. Must be all lower case.
            - to: the city where the travel ends. Must be all lower case.
            - options: set this to []
        The query will return:
            - 'False' if there are no routes availables
            - A dictionary with 'Results' as the key and a list of possible routes as the value
        Do not use quotes.
    """,
)

## Invocation
If a schema is defined, we can pass a dictionary using the names of the parameters in the schema as the keys in the dictionary. The values can represent Prolog variables (uppercase first letter) or strings (lower case first letter). A `None` value is interpreted as a variable and replaced with the key capitalized:

In [ ]:
planner_tool.invoke(
    {
        'query_type': 'all',
        'from': 'paris',
        'to': 'lisbon',
        'options': [],
        'results': None,
    }
)

## Setting up the Chat Model

In [ ]:
#@title Run this cell to instantiate the Chat model. Click on > to view/hide the script. {display-mode: "form"}

%%capture --no-stderr --no-display

from IPython.display import display, HTML

if provider == "OpenAI":

  from langchain_openai import ChatOpenAI

  model = "gpt-4o-mini"
  llm = ChatOpenAI(model=model, temperature=0)
  max_tries = 10
  get_answer = lambda response: response["output"]

elif provider == "Anthropic":

  from langchain_anthropic import ChatAnthropic

  model = "claude-3-5-sonnet-latest"
  llm = ChatAnthropic(model=model, temperature=0)
  max_tries = 10
  get_answer = lambda response: response["output"][0]["text"]
  model = llm.model

elif provider == "Ollama":

  from langchain_ollama import ChatOllama

  model = "llama3.1:8b"
  llm = ChatOllama(
          model=model,
          base_url="http://localhost:11434",
          temperature=0
        )
  max_tries = 100
  get_answer = lambda response: response["output"]
  model = llm.model

llm.invoke("Hi")
display(HTML(f"<div style='padding: 10px; background-color: #d4edda; border-radius: 5px; text-align: center;'><b>Success!</b> {model} instantiated!</div>"))

## Set up your own query

In [ ]:
#@title Run this cell to set your travel query. Click on > to view/hide the script. {display-mode: "form"}

from ipywidgets import widgets
from IPython.display import display

cities = ['London', 'Paris', 'Rome', 'Athens', 'Barcelona', 'Madrid', 'Lisbon']
from_city = widgets.Dropdown(options=cities, description='From:')
to_city = widgets.Dropdown(options=cities, description='To:')
query_type = widgets.Dropdown(options=['all', 'fastest', 'cheapest'], description='Query:')

display(from_city, to_city, query_type)

def on_button_clicked(b):
    global query
    #query = f"What's the {query_type.value} route from {from_city.value} to {to_city.value}?"
    query = f"I want to tavel from from {from_city.value} to {to_city.value}. "
    if query_type.value == 'fastest':
        query = query + "What's the fastest route?"
    elif query_type.value == 'cheapest':
        query = query + "What's the cheapest route?"
    else:
        query = query + "What are my options?"
    print(f"\n{query}")
    query = f"""
            Always use the prolog_travel_planner tool for any questions
            regarding travelling, respond with a detailed answer using all
            the information provided by the prolog_travel_planner tool.
            Don't try to use your own knowledge or real time data.

            Question: {query}
            """

button = widgets.Button(description="Define Query")
button.on_click(on_button_clicked)
display(button)

## Using an LLM and function calling

Yor can use any LangChain chat model that supports tool calling. To use the Prolog tool, bind it to the chat model:

In [ ]:
llm_with_tools = llm.bind_tools([planner_tool])

and then query the model:

In [ ]:
tries = 1
while tries <= max_tries:
  try:

    # The user query is pass in as a list of messages
    messages = [("human", query)]
    response = llm_with_tools.invoke(messages)

    # The LLM will respond with a tool call request if needed
    if hasattr(response, 'tool_calls') and response.tool_calls:
        messages.append(response)

        # The tool takes this request and queries the Prolog database:
        tool_msg = planner_tool.invoke(response.tool_calls[0])
        messages.append(tool_msg)

        #The tool returns a list with all the solutions for the query
        response = llm_with_tools.invoke(messages)

    #Check that we have a valid response
    if response.content:
      show_ai_message(response.content)
      break
    else:
      tries += 1

  except Exception as e:
    # The LLM failed to communicate with the tool using the right schema. It happens!
    tries += 1

if tries > max_tries:
  show_ai_message("Could not get an answer", height=100)

## Using a LangChain Agent
To use a Prolog tool with an agent, pass it to the agent's constructor:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

prompt = ChatPromptTemplate.from_messages(
    [
      ("system", "You are a helpful assistant"),
      ("human", "{input}"),
      ("placeholder", "{agent_scratchpad}"),
    ]
)
tools = [planner_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

The agent takes the query and use the Prolog tool if needed. Then the agent receives​ the tool response and generates the answer:

In [ ]:
tries = 1
while tries <= max_tries:
  try:
    response = agent_executor.invoke({"input": query})
    answer = get_answer(response)
    show_ai_message(answer)
    break
  except Exception as e:
    tries += 1
if tries > max_tries:
  show_ai_message("Could not get an answer", height=100)

## Using a LangGraph Agent
To use a Prolog tool with an agent, pass it to the agent's constructor:

In [ ]:
from langgraph.prebuilt import create_react_agent

tools = [planner_tool]
lg_agent = create_react_agent(llm, tools)

The agent takes the query and use the Prolog tool if needed. Then the agent receives​ the tool response and generates the answer:

In [ ]:
verbose = False
tries = 1
inputs = {"messages": [("human", query)]}
while tries <= max_tries:
  try:
    if verbose:
        for step in lg_agent.stream(inputs, stream_mode="values"):
            message = step["messages"][-1]
            message.pretty_print()
    else:
        outputs = lg_agent.invoke(inputs)
        results = outputs["messages"][-1]
        results.pretty_print()
    break
  except Exception as e:
    tries += 1
if tries > max_tries:
    show_ai_message("Could not get an answer", height=100)

## API reference

See https://langchain-prolog.readthedocs.io/en/latest/modules.html for detail.